# Import libraries

In [1]:
import os 
from imutils import paths 
import cv2 
import numpy as np 
import matplotlib.pyplot as plt 
from skimage import feature 
from sklearn.svm import LinearSVC

# path of training and test files

In [2]:
training_data_path = 'dataset/training'
test_data_path = 'dataset/test'

# paramters for LBPH method

In [3]:
numPoints = 24
radius = 8

# prepare training data by extracting LBPH features from each image

In [4]:
train_data = []
train_labels = []
eps = 1e-7
for image_path in paths.list_images(training_data_path):
    # read the image, convert it to grayscale
    image = cv2.imread(image_path)
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    # extract LBPH features, method is uniform means feature extraction is rotation invariance    
    lbp_features = feature.local_binary_pattern(gray, numPoints, radius, method="uniform")
    (hist, _) = np.histogram(lbp_features.ravel(), bins=np.arange(0, numPoints + 3), range=(0, numPoints + 2))
    hist = hist.astype("float")
    # normalize histograms
    hist /= (hist.sum() + eps)
    #get the label from the image path, append in a list
    train_data.append(hist)
    train_labels.append(image_path.split(os.path.sep)[-2])

# train a linear SVM model

In [5]:
# train a Linear SVM on the data
model = LinearSVC(C=70, random_state=42)
model.fit(train_data, train_labels)

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\svm\_base.py:947: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


LinearSVC(C=70, class_weight=None, dual=True, fit_intercept=True,
          intercept_scaling=1, loss='squared_hinge', max_iter=1000,
          multi_class='ovr', penalty='l2', random_state=42, tol=0.0001,
          verbose=0)

# test the model on the test images

In [6]:
test_data = []
prediction_result = []
for image_path in paths.list_images(test_data_path):
    # read the image, convert it to grayscale
    image = cv2.imread(image_path)
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    # Extract LBPH features
    lbp = feature.local_binary_pattern(gray, numPoints, radius, method="uniform")
    (hist, _) = np.histogram(lbp.ravel(), bins=np.arange(0, numPoints + 3), range=(0, numPoints + 2))
    hist = hist.astype("float")
    # Normalize histograms
    hist /= (hist.sum() + eps)
    test_data.append(hist)
    prediction = model.predict(hist.reshape(1, -1))
    # display the image and the prediction
    cv2.putText(image, prediction[0], (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255))
    cv2.imshow("predicted image", image)
    cv2.waitKey(5000)
cv2.destroyAllWindows()